In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import itertools
import sys
import os
import math
sys.path.append('../..')
import scipy.optimize as opt
from joblib import Parallel, delayed
from functools import partial
from hyperopt import hp, tpe, fmin, Trials
from models.optim.sir_simple import SEIR

In [2]:
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def calculate_opt(intervention_day, intervention_duration, intervention_choice, days):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    
    params = [T_trans, T_treat, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    
    coeficeint=[S_coeficeint,I_coeficeint,R_coeficeint]
    
    for i in range(days):
        grad1 = np.dot(coeficeint,np.sum(states_int_array/1e6, axis=1))
    return(grad1)

In [3]:
fx = calculate_opt(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=400)
print(fx)

1.4999090000000002


In [4]:
def check(start_array, duration_array, choice_array, total_resource):
    for i in range(1,len(start_array)):
        if(start_array[i] <= start_array[i-1] + duration_array[i-1]):
            return(0)
    resource_spent = np.dot(duration_array, choice_array)
    if(resource_spent != total_resource):
        return(0)
    else:
        return(1)
    

def grid_search(num_int, total_resource=30, day0=30):
    min_val = 100
    min_params = {}
    params = []
    min_duration = 10
    max_duration = total_resource*np.array([4,2,1])
   
    if(num_int == 1):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val))
        choice_tuple = list(itertools.product(choice_val))
        duration_tuple = list(itertools.product(duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
    if(num_int == 2):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
            
    if(num_int == 3):
        start_val = [start for start in range(day0, day0+365, 40)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 10)]
        start_tuple = list(itertools.product(start_val, start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
                            
    print(len(params))

    value_array = Parallel(n_jobs=40)(delayed(calculate_opt)(intervention_day=par[0], intervention_duration=par[2],\
                                                             intervention_choice=par[1], days = 400) for par in params)
    value_array = np.array(value_array)
    min_val = np.min(value_array)
    i = np.argmin(value_array)
    min_params['start_array'] = params[i][0]
    min_params['duration_array'] = params[i][2]
    min_params['choice_array'] = params[i][1]
    
    return(min_val, min_params)    

In [5]:
#num_int = 1
grid_search(num_int=1)

74


(1.4997589999999998,
 {'start_array': array([30]),
  'duration_array': array([30]),
  'choice_array': array([1])})

In [6]:
# num_int = 2
grid_search(num_int=2)

36667


(1.4997159999999998,
 {'start_array': array([30, 50]),
  'duration_array': array([15, 15]),
  'choice_array': array([1, 1])})

In [7]:
#num_int = 3
grid_search(num_int=3)

23048


(1.499777,
 {'start_array': array([ 30,  70, 110]),
  'duration_array': array([20, 10, 10]),
  'choice_array': array([1. , 0.5, 0.5])})

In [8]:
def check(start_array, duration_array, choice_array, total_resource):
    for i in range(1,len(start_array)):
        if(start_array[i] <= start_array[i-1] + duration_array[i-1]):
            return(0)
    resource_spent = np.dot(duration_array, choice_array)
    if(resource_spent != total_resource):
        return(0)
    else:
        return(1)

# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def mod_calculate_opt(variable_params, total_resource, days):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0

    intervention_day = np.array(variable_params['intervention_day'])
    intervention_duration = np.array(variable_params['intervention_duration'])
    intervention_choice = np.array(variable_params['intervention_choice'])

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    if(not check(intervention_day, intervention_duration, intervention_choice, total_resource)):
        return(100)
    
    params = [T_trans, T_treat, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
 
    coeficeint=[S_coeficeint,I_coeficeint,R_coeficeint]
    
    for i in range(days):
        grad1 = np.dot(coeficeint,np.sum(states_int_array/1e6, axis=1))
        
    return(grad1)

def tpe_opt(num_int, total_resource=30, day0=30):
    if(num_int==1):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource+1, 5)]

        variable_params = {
            'intervention_day' : [hp.choice('intervention_day', start_val)],
            'intervention_duration' : [hp.choice('intervention_duration', duration_val)],
            'intervention_choice' : [hp.choice('intervention_choice', choice_val)],
        }
        
    if(num_int==2):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource+1, 5)]

        variable_params = {
            'intervention_day' : [hp.choice('id_0', start_val),hp.choice('id_1', start_val)],
            'intervention_duration' : [hp.choice('du_0', duration_val),hp.choice('du_1', duration_val)],
            'intervention_choice' : [hp.choice('ic_0', choice_val),hp.choice('ic_1', choice_val)],
        }

    partial_calculate_opt = partial(mod_calculate_opt, total_resource=total_resource, days=400) 
    
    searchspace = variable_params
    
    trials = Trials()
    best = fmin(partial_calculate_opt,
                space=searchspace,
                algo=tpe.suggest,
                max_evals=3000,
                trials=trials)
    
    return(best, trials)

In [9]:
tpe_opt(num_int=1)

100%|██████████| 3000/3000 [01:01<00:00, 48.76trial/s, best loss: 1.4997589999999998]


({'intervention_choice': 2, 'intervention_day': 0, 'intervention_duration': 4},
 <hyperopt.base.Trials at 0x7f0f2ff8a0b8>)

In [10]:
tpe_opt(num_int=2)

100%|██████████| 3000/3000 [01:35<00:00, 31.44trial/s, best loss: 1.499819]          


({'du_0': 5, 'du_1': 8, 'ic_0': 1, 'ic_1': 0, 'id_0': 0, 'id_1': 4},
 <hyperopt.base.Trials at 0x7f0f2fe01048>)

In [11]:
from models.optim.sir_cont import SEIR
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def run_seir(days, int_vec):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0
    
    params = [T_trans, T_treat, N, int_vec]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    derivatives = np.ones((days,3))
    for t in range(days):
        derivatives[t] = solver.get_derivative(t=t, y=states_int_array[:,t])
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    coeficeint=np.array([S_coeficeint,I_coeficeint,R_coeficeint])
    
    grad1 = np.ones(days)
    grad2 = np.ones(days)

    for i in range(days):
        grad2[i] = np.dot(coeficeint,(derivatives[i]/1e6))
        grad1[i] = np.dot(coeficeint,(states_int_array[:,i]/1e6))
    return(grad1)

In [12]:
total_resource = 30
resource_spent = 0
window = 5
int_vec = np.ones(400)
while(resource_spent<total_resource):
    curve = run_seir(days=400, int_vec=int_vec)
    arg_curve = (-curve).argsort()
    day_max = arg_curve[0]
    i=1
    while(int_vec[day_max]>=2):
        day_max = arg_curve[i]
        i += 1

    int_vec[day_max] += 0.25 
    resource_spent += 0.25

In [13]:
int_vec

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [14]:
curve = run_seir(days=400, int_vec=int_vec)
np.sum(curve)

1.49848

In [15]:
curve_base = run_seir(days = 400, int_vec=np.ones(400))
np.sum(curve_base)

1.4999090000000002